In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam
# importing the tools required for the Poisson regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import minimize

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
MLS2020 = pd.read_csv("https://www.football-data.co.uk/new/USA.csv")
#Swed2020 = pd.read_csv("https://www.football-data.co.uk/new/SWE.csv")
Nor2020 = pd.read_csv("https://www.football-data.co.uk/new/NOR.csv")
#LigaMx2020 = pd.read_csv("https://www.football-data.co.uk/new/MEX.csv")
#LigaMx2020 = LigaMx2020[LigaMx2020['Season'] == "2020/2021"]
Other_2020 = pd.concat([Nor2020,MLS2020],sort = False)
#Other_2020 = pd.concat([Swed2020],sort = False)
Other_2020 = Other_2020[Other_2020['Season'] == 2021]
#Other_2020 = pd.concat([Other_2020, LigaMx2020])
Other_2020 = Other_2020[['Home','Away','HG','AG']]
Other_2020 = Other_2020.rename(columns={'Home': 'HomeTeam', 'Away': 'AwayTeam'})
Other_2020 = Other_2020.rename(columns={'HG': 'FTHG', 'AG': 'FTAG'})

epl_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
epl_1819 = epl_1920
bundes_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/D1.csv")
ecl_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E1.csv")
serieA_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/I1.csv")
laliga_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/SP1.csv")
ligue1_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/F1.csv")
eredivise_1920 = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/N1.csv")
#epl_1819 = pd.concat([epl_1920, bundes_1920, ecl_1920, serieA_1920, laliga_1920, ligue1_1920, eredivise_1920, Other_2020],sort = False)
epl_1819 = pd.concat([epl_1920, bundes_1920, ecl_1920, serieA_1920, laliga_1920, eredivise_1920],sort = False)
#epl_1819 = pd.concat([epl_1819,ligue1_1920], sort = False)
#epl_1819 = pd.concat([epl_1819,ligue1_1920,Other_2020], sort = False)
#epl_1819 = pd.concat([epl_1920,bundes_1920,ecl_1920,ligue1_1920], sort = False)

epl_1819 = epl_1819[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl_1819 = epl_1819.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals'})


'''print("List of Teams, it is case sensitive and type it without Quotes!")'''
print('im finally fucking onde')
print(np.sort(epl_1819.HomeTeam.unique()))

#print(epl_1819.tail(10))
#print(epl_1819["HomeTeam"].count())
#print(epl_1819["AwayTeam"].count())


im finally fucking onde
['AZ Alkmaar' 'Ajax' 'Alaves' 'Arsenal' 'Aston Villa' 'Atalanta'
 'Ath Bilbao' 'Ath Madrid' 'Augsburg' 'Barcelona' 'Barnsley'
 'Bayern Munich' 'Benevento' 'Betis' 'Bielefeld' 'Birmingham' 'Blackburn'
 'Bologna' 'Bournemouth' 'Brentford' 'Brighton' 'Bristol City' 'Burnley'
 'Cadiz' 'Cagliari' 'Cardiff' 'Celta' 'Chelsea' 'Coventry' 'Crotone'
 'Crystal Palace' 'Den Haag' 'Derby' 'Dortmund' 'Eibar' 'Ein Frankfurt'
 'Elche' 'Everton' 'FC Emmen' 'FC Koln' 'Feyenoord' 'Fiorentina'
 'For Sittard' 'Freiburg' 'Fulham' 'Genoa' 'Getafe' 'Granada' 'Groningen'
 'Heerenveen' 'Heracles' 'Hertha' 'Hoffenheim' 'Huddersfield' 'Huesca'
 'Inter' 'Juventus' 'Lazio' 'Leeds' 'Leicester' 'Levante' 'Leverkusen'
 'Liverpool' 'Luton' "M'gladbach" 'Mainz' 'Man City' 'Man United'
 'Middlesbrough' 'Milan' 'Millwall' 'Napoli' 'Newcastle' 'Norwich'
 "Nott'm Forest" 'Osasuna' 'PSV Eindhoven' 'Parma' 'Preston' 'QPR'
 'RB Leipzig' 'Reading' 'Real Madrid' 'Roma' 'Rotherham' 'Sampdoria'
 'Sassuolo' 

In [3]:
goal_model_data = pd.concat([epl_1819[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
		   epl_1819[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, family=sm.families.Poisson()).fit()




def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
	home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 'opponent': awayTeam,'home':1},index=[1])).values[0]

	away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 'opponent': homeTeam,'home':0},index=[1])).values[0]

	team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]

	return (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

def print_match(simulation):
	#Home win
	a=np.sum(np.tril(simulation, -1))
	#Draw
	b=np.sum(np.diag(simulation))
	#Away WinWin
	c=np.sum(np.triu(simulation, 1))
	maximum = max(a,b,c)
	if maximum == a:
	#	print(f"Home wins @ {a}%")
	#	if b > c:
		#	print(f"Draw 2nd @ {b}")
	#	elif c > b:
		#	print(f"Away 2nd @ {c}")

		return "Home"
	elif maximum == b:
		#print(f"Draw Wins @ {b}%")
		#if a > c:
		#	print(f"Home 2nd @ {a}")
		#elif c > a:
			#print(f"Away 2nd @ {c}")

		return "Draw"
	elif maximum == c:
		#print(f"Away Wins @ {c}%")
		#if a > b:
			#print(f"Home 2nd @ {a}")
		#elif b > a:
			#print(f"Draw 2nd @ {b}")
		return "Away"

def print_match_2(simulation):
	#Home win
	a=np.sum(np.tril(simulation, -1))
	#Draw
	b=np.sum(np.diag(simulation))
	#Away Win
	c=np.sum(np.triu(simulation, 1))
	maximum = max(a,b,c)
	return maximum

def print_match_3(simulation):
	#Home win
	a=np.sum(np.tril(simulation, -1))
	#Draw
	b=np.sum(np.diag(simulation))
	#Away Win
	c=np.sum(np.triu(simulation, 1))
	maximum = max(a,b,c)
	return f"{a:.4f}, {b:.4f}, {c:.4f}"

def match_handicap(match, Home = True, Handicap = 1.5):
	count = 0
	for i in range(11):
		for j in range(11):
			if Home == True:
				if i - j >= Handicap:
					count += match[i][j]
			elif Home == False:
				if j - i >= Handicap:
					count+= match[i][j]
	print(count)
	return count

def match_ou(match, Home = True, ou = 2.5, text = "Over"):
	count = 0
	for i in range(11):
		for j in range(11):
			if text == "Over" or text == 'over':
				if Home == True:
					if i + j >= ou:
						count += match[i][j]
				elif Home == False:
					if j + i >= ou:
						count+= match[i][j]
		#	elif text == "Under" or text == 'under':
			#	if Home == True:
			#		if i + j <= ou:
					#	count += match[i][j]
			#	elif Home == False:
				#	if j + i <= ou:
					#	count+= match[i][j]
	return count

def PTS(simulation):
	PTS = []
	highest_prob = 0
	home_score = 0
	away_score = 0
	for i in range(6):
		for j in range (6):
			if simulation[i][j] < highest_prob:
				continue
			else:
				highest_prob = simulation[i][j]
				home_score = i
				away_score = j

	print(f"Probability of {home_score} - {away_score} happening: {highest_prob}")
	PTS.append(home_score)
	PTS.append(away_score)
	return PTS

def PTS_1(simulation):
	PTS = []
	highest_prob = 0
	home_score = 0
	away_score = 0
	for i in range(6):
		for j in range (6):
			if simulation[i][j] < highest_prob:
				continue
			else:
				highest_prob = simulation[i][j]
				home_score = i
				away_score = j

	#print(f"Probability of {home_score} - {away_score} happening: {highest_prob}")
	#PTS.append(home_score)
	#PTS.append(away_score)
	return highest_prob

In [4]:

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))
def solve_parameters(dataset, debug = False, init_vals=None, options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['HomeTeam'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['AwayTeam'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                     ))
    def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
        lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
        return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
                np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        log_like = [dc_log_like(row.HomeGoals, row.AwayGoals, score_coefs[row.HomeTeam], defend_coefs[row.HomeTeam],
                     score_coefs[row.AwayTeam], defend_coefs[row.AwayTeam], rho, gamma) for row in dataset.itertuples()]
        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))
#params = solve_parameters(epl_1819)

##Update Params at 8 August 2020 W/ MLS
#params

In [5]:
value = 100
total_games = 0
win_games = 0

big_win_games = 0
small_win_games = 0
medium_win_games = 0

total_big_games = 0
total_small_games = 0
total_medium_games = 0


big_threshold = 0.7
small_threshold = 0.4

big_list = []
big_count_list = []
small_list = []
small_count_list = []


#for i in range(70,90):
for i in range(1):
	#big_threshold = i/100
	#print("big is now:", big_threshold)
	for j in range(1):
	#for j in range(45,50):
		#if j < i:
		#small_threshold = j/100
		#else:
			#continue

		print("small is now:", small_threshold)

		big_win_games = 0
		small_win_games = 0
		medium_win_games = 0

		total_big_games = 0
		total_small_games = 0
		total_medium_games = 0

		for index, row in epl_1819.iterrows():
			home_team = row['HomeTeam']
			home_goals = row['HomeGoals']
			away_team = row['AwayTeam']
			away_goals = row["AwayGoals"]
			#sim = simulate_match(home_team, away_team)
			try:
				sim = simulate_match(poisson_model, home_team, away_team, max_goals=10)
			except:
				continue
	    

			if print_match_2(sim) >= big_threshold:
				value_mult = 0.5
				total_big_games += 1
			elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
				value_mult = 0
				total_medium_games += 1
			elif print_match_2(sim) < small_threshold:
				value_mult = 0
				total_small_games += 1
				
			if home_goals > away_goals:
				if print_match(sim) == "Home":
					value += value_mult
					win_games += 1
					if print_match_2(sim) >= big_threshold:
						big_win_games += 1
					elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
						medium_win_games += 1
					elif print_match_2(sim) < small_threshold:
						small_win_games += 1
				else:
					value -= value_mult*5
				total_games += 1
			elif away_goals > home_goals:
				if print_match(sim) == "Away":
					value += value_mult
					win_games += 1 
					if print_match_2(sim) >= big_threshold:
						big_win_games += 1
					elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
						medium_win_games += 1
					elif print_match_2(sim) < small_threshold:
						small_win_games += 1
				else:
					value -= value_mult
				total_games += 1
			elif away_goals == home_goals:
				if print_match(sim) == "Draw":
					value += value_mult
					win_games += 1 
					if print_match_2(sim) >= big_threshold:
						big_win_games += 1
					elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
						medium_win_games += 1
					elif print_match_2(sim) < small_threshold:
						small_win_games += 1
						
				else:
					value -= value_mult
				total_games += 1
		try:
			big_list.append(big_win_games/total_big_games)
			big_count_list.append(big_threshold)

			small_list.append(small_win_games/total_small_games)
			small_count_list.append(small_threshold)
		except:
			pass



	print(f'Value = {value}')
	print(f'Won games = {win_games}, Total Games = {total_games}, {(win_games*100)/total_games}% Accuracy')
	#print(f'Won big games = {big_win_games}, Total Games = {total_big_games}, {(big_win_games*100)/total_big_games}% Accuracy')
	#print(f'Won small games = {small_win_games}, Total Games = {total_small_games}, {(small_win_games*100)/total_small_games}% Accuracy')
	#print(f'Won medium games = {medium_win_games}, Total Games = {total_medium_games}, {(medium_win_games*100)/total_medium_games}% Accuracy')

small is now: 0.4
Value = 173.0
Won games = 1237, Total Games = 2304, 53.689236111111114% Accuracy


In [6]:
total_ou_game = 0
win_ou_game = 0 

total_ou_big = 0
win_ou_big = 0

total_ou_small = 0
win_ou_small = 0

total_ou_custom = 0
win_ou_custom = 0

for index, row in epl_1819.iterrows():

	home_team = row['HomeTeam']
	home_goals = row['HomeGoals']
	away_team = row['AwayTeam']
	away_goals = row["AwayGoals"]
  #sim = simulate_match(home_team, away_team)
	try:
		sim = simulate_match(poisson_model, home_team, away_team, max_goals=10)
	except:
		continue
	match_over = match_ou(sim)
	actual_ou = home_goals + away_goals
	total_ou_game += 1

	if match_over < 0.5:
		total_ou_small += 1
		if actual_ou < 2.5:
			win_ou_small += 1

	if match_over >= 0.5 and actual_ou > 2.5:
		win_ou_game += 1

	if match_over >= 0.7:
		total_ou_big += 1
		if actual_ou >2.5:
			win_ou_big += 1

	if match_over >= 0.35 and match_over < 0.4:
		total_ou_custom += 1
		if actual_ou <= 2.5:
			win_ou_custom += 1
	#elif match_over < 0.5 and actual_ou < 2.5:
		#win_ou_game += 1

print(f"Over/Under accuracy: {win_ou_game*100 / total_ou_game}%")
print(f"Over/Under accuracy less than 0.5: {win_ou_small*100 / total_ou_small}%;; {win_ou_small} over {total_ou_small}")
#print(f"Over/Under accuracy more than 0.7: {win_ou_big*100 / total_ou_big}%;; {win_ou_big} over {total_ou_big}")
#print(f"Over/Under accuracy for custom: {win_ou_custom*100 / total_ou_custom}%;; {win_ou_custom} over {total_ou_custom}")

Over/Under accuracy: 29.296875%
Over/Under accuracy less than 0.5: 59.966358284272495%;; 713 over 1189


In [7]:
overall_win_x_axis = []
for i in range(35,95,5):
  overall_win_x_axis.append(i/100)

In [8]:
overall_win_x_axis

[0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]

In [ ]:
overall_actual_win_y = []

for i in overall_win_x_axis:
  big_win_games = 0
  small_win_games = 0
  medium_win_games = 0
  total_big_games = 0
  total_small_games = 0
  total_medium_games = 0
  big_threshold = i+0.05
  small_threshold = i
  value = 100
  print(big_threshold)
  
  for index, row in epl_1819.iterrows():
    home_team = row['HomeTeam']
    home_goals = row['HomeGoals']
    away_team = row['AwayTeam']
    away_goals = row["AwayGoals"]
    #sim = simulate_match(home_team, away_team)
    try:
      sim = simulate_match(poisson_model, home_team, away_team, max_goals=10)    
    except:
      continue    
    
    if print_match_2(sim) >= big_threshold:
      total_big_games += 1
    elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
      total_medium_games += 1
    elif print_match_2(sim) < small_threshold:
      total_small_games += 1
    
    if home_goals > away_goals:
      if print_match(sim) == "Home":
        value += value_mult
        win_games += 1
        if print_match_2(sim) >= big_threshold:
          big_win_games += 1
        elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
          medium_win_games += 1
        elif print_match_2(sim) < small_threshold:
          small_win_games += 1
      else:
        value -= value_mult
      total_games += 1
    elif away_goals > home_goals:
      if print_match(sim) == "Away":
        value += value_mult
        win_games += 1 
        if print_match_2(sim) >= big_threshold:
          big_win_games += 1
        elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
          medium_win_games += 1
        elif print_match_2(sim) < small_threshold:
          small_win_games += 1
      else:
        value -= value_mult
      total_games += 1
    elif away_goals == home_goals:
      if print_match(sim) == "Draw":
        value += value_mult
        win_games += 1 
        if print_match_2(sim) >= big_threshold:
          big_win_games += 1
        elif print_match_2(sim) < big_threshold and print_match_2(sim) >= small_threshold:
          medium_win_games += 1
        elif print_match_2(sim) < small_threshold:
          small_win_games += 1
          
      else:
        value -= value_mult
      total_games += 1
  try:
    big_list.append(big_win_games/total_big_games)
    big_count_list.append(big_threshold)
    small_list.append(small_win_games/total_small_games)
    small_count_list.append(small_threshold)
  except:
    pass

  try:
    overall_actual_win_y.append((medium_win_games*100)/total_medium_games)
  except:
    overall_actual_win_y.append(0)

0.39999999999999997


In [ ]:
overall_actual_win_y

In [ ]:
plt.plot(overall_win_x_axis,overall_actual_win_y,".")

In [ ]:
overall_win_x_axis_array = np.array(overall_win_x_axis).reshape((-1, 1))
overall_actual_win_y_array = np.array(overall_actual_win_y)
overall_win_model = LinearRegression().fit(overall_win_x_axis_array,overall_actual_win_y_array)
r_sq = overall_win_model.score(overall_win_x_axis_array, overall_actual_win_y_array)
print('coefficient of determination:', r_sq)

In [ ]:
plt.plot(overall_win_x_axis,overall_actual_win_y,".")
plt.plot(overall_win_x_axis_array, overall_win_model.predict(overall_win_x_axis_array))
plt.plot(overall_win_x_axis, np.array(overall_win_x_axis)*100)
plt.show()


In [ ]:
print(overall_win_x_axis)
print(overall_actual_win_y)

In [ ]:
import csv

with open('overall_win.csv', mode = 'w', newline='\n') as file:
	Fwriter = csv.writer(file, delimiter=',')
	Fwriter.writerow(['x']+(['y']))
	count = 0
	for index in range(len(overall_win_x_axis)):
		Fwriter.writerow([overall_win_x_axis[index]] + [overall_actual_win_y[index]])
		count += 1
		print(count)
print("Done~")


In [ ]:
overall_ou_x_axis = []
for i in range(15,95,5):
  overall_ou_x_axis.append(i/100)

In [ ]:
'''overall_ou_win_y = []
for i in overall_ou_x_axis:
  total_ou_game = 0
  win_ou_game = 0 

  total_ou_big = 0
  win_ou_big = 0

  total_ou_small = 0
  win_ou_small = 0

  total_ou_custom = 0
  win_ou_custom = 0
  ou_big_thresh = i +0.05
  ou_small_thresh = i
  print(ou_big_thresh)

  for index, row in epl_1819.iterrows():

    home_team = row['HomeTeam']
    home_goals = row['HomeGoals']
    away_team = row['AwayTeam']
    away_goals = row["AwayGoals"]
    sim = simulate_match(poisson_model, home_team, away_team, max_goals=10)
    match_over = match_ou(sim)
    actual_ou = home_goals + away_goals
    total_ou_game += 1

    if match_over >= ou_small_thresh and match_over < ou_big_thresh:
      total_ou_custom += 1
      #if actual_ou <= 2.5 and match_over < 0.5:
        #win_ou_custom += 1
      #elif actual_ou >= 2.5 and match_over > 0.5:
      if actual_ou >= 2.5:
        win_ou_custom += 1
    #elif match_over < 0.5 and actual_ou < 2.5:
      #win_ou_game += 1
  try:
    overall_ou_win_y.append(win_ou_custom*100 / total_ou_custom)
  except:
    continue
'''

In [ ]:
overall_ou_win_y = []
ou_win_y_2 = []
ou_win_y_25 = []
ou_win_y_3 = []
ou_win_y_35 = []
ou_win_y_4 = []
ou_win_y_45 = []
ou_win_y_5 = []
ou_win_y_55 = []
ou_win_y_6= []
ou_win_y_65 = []
ou_win_y_7 = []
ou_win_y_75 = []
ou_win_y_8 = []
ou_win_y_85 = []
ou_win_y_9 = []
ou_win_y_95 = []

for index, row in epl_1819.iterrows():

  home_team = row['HomeTeam']
  home_goals = row['HomeGoals']
  away_team = row['AwayTeam']
  away_goals = row["AwayGoals"]
  #sim = simulate_match(home_team, away_team)
  sim = simulate_match(poisson_model, home_team, away_team, max_goals=10)
  match_over = match_ou(sim)
  actual_ou = home_goals + away_goals
  total_ou_game += 1
  if match_over >= 0.15 and match_over < 0.2:
    if actual_ou >= 2.5:
      ou_win_y_2.append(1)
    else:
      ou_win_y_2.append(0)

  elif match_over >= 0.2 and match_over < 0.25:
    if actual_ou >= 2.5:
      ou_win_y_25.append(1)
    else:
      ou_win_y_25.append(0)

  elif match_over >= 0.25 and match_over < 0.3:
    if actual_ou >= 2.5:
      ou_win_y_3.append(1)
    else:
      ou_win_y_3.append(0)

  elif match_over >= 0.3 and match_over < 0.35:
    if actual_ou >= 2.5:
      ou_win_y_35.append(1)
    else:
      ou_win_y_35.append(0)

  elif match_over >= 0.35 and match_over < 0.4:
    if actual_ou >= 2.5:
      ou_win_y_4.append(1)
    else:
      ou_win_y_4.append(0)

  elif match_over >= 0.4 and match_over < 0.45:
    if actual_ou >= 2.5:
      ou_win_y_45.append(1)
    else:
      ou_win_y_45.append(0)

  elif match_over >= 0.45 and match_over < 0.5:
    if actual_ou >= 2.5:
      ou_win_y_5.append(1)
    else:
      ou_win_y_5.append(0)

  elif match_over >= 0.5 and match_over < 0.55:
    if actual_ou >= 2.5:
      ou_win_y_55.append(1)
    else:
      ou_win_y_55.append(0)

  elif match_over >= 0.55 and match_over < 0.6:
    if actual_ou >= 2.5:
      ou_win_y_6.append(1)
    else:
      ou_win_y_6.append(0)

  elif match_over >= 0.6 and match_over < 0.65:
    if actual_ou >= 2.5:
      ou_win_y_65.append(1)
    else:
      ou_win_y_65.append(0)
  elif match_over >= 0.65 and match_over < 0.7:
    if actual_ou >= 2.5:
      ou_win_y_7.append(1)
    else:
      ou_win_y_7.append(0)

  elif match_over >= 0.7 and match_over < 0.75:
    if actual_ou >= 2.5:
      ou_win_y_75.append(1)
    else:
      ou_win_y_75.append(0)

  elif match_over >= 0.75 and match_over < 0.8:
    if actual_ou >= 2.5:
      ou_win_y_8.append(1)
    else:
      ou_win_y_8.append(0)

  elif match_over >= 0.8 and match_over < 0.85:
    if actual_ou >= 2.5:
      ou_win_y_85.append(1)
    else:
      ou_win_y_85.append(0)

  elif match_over >= 0.85 and match_over < 0.9:
    if actual_ou >= 2.5:
      ou_win_y_9.append(1)
    else:
      ou_win_y_9.append(0)

  elif match_over >= 0.9 and match_over < 0.95:
    if actual_ou >= 2.5:
      ou_win_y_95.append(1)
    else:
      ou_win_y_95.append(0)

def average_func(lst):
  try:
    overall_ou_win_y.append((sum(lst)/len(lst))*100)
  except:
    overall_ou_win_y.append(0)

average_func(ou_win_y_2)
average_func(ou_win_y_25)
average_func(ou_win_y_3)
average_func(ou_win_y_35)
average_func(ou_win_y_4)
average_func(ou_win_y_45)
average_func(ou_win_y_5)
average_func(ou_win_y_55)
average_func(ou_win_y_6)
average_func(ou_win_y_65)
average_func(ou_win_y_7)
average_func(ou_win_y_75)
average_func(ou_win_y_8)
average_func(ou_win_y_85)
average_func(ou_win_y_9)
average_func(ou_win_y_95)

In [ ]:
overall_ou_x_axis

In [ ]:
overall_ou_win_y

In [ ]:
plt.plot(overall_ou_x_axis,overall_ou_win_y,".")

In [ ]:
overall_ou_x_axis_array = np.array(overall_ou_x_axis).reshape((-1,1))
overall_ou_win_y_array = np.array(overall_ou_win_y)
overall_ou_model = LinearRegression().fit(overall_ou_x_axis_array, overall_ou_win_y_array)
r_sq = overall_ou_model.score(overall_ou_x_axis_array, overall_ou_win_y_array)
print('coefficient of determination:', r_sq)

In [ ]:
plt.plot(overall_ou_x_axis,overall_ou_win_y,".")
plt.plot(overall_ou_x_axis,overall_ou_model.predict(overall_ou_x_axis_array))
plt.plot(overall_win_x_axis, np.array(overall_win_x_axis)*100)
plt.show()

In [ ]:
def kelly(bm_odds,percentage_win,bankroll = 100):
  percentage_win = percentage_win/100
  return percentage_win - (1-percentage_win)/(bm_odds-1)

In [ ]:
def final_output(Home_team, Away_team):
  match = simulate_match(poisson_model, Home_team, Away_team)
  #match = simulate_match(Home_team, Away_team)
  input_predict = np.array(print_match_2(match)).reshape((-1, 1))
  input_predict_ou = np.array(match_ou(match)).reshape((-1,1))
  ou_transformed = PolynomialFeatures(degree=2, include_bias=False).fit_transform(input_predict_ou)
  print(PTS(match))
  #print(match_ou(match))
  print(print_match(match))#, print_match_2(match), f"Implied odds: {1/print_match_2(match)}")
  #print(print_match_3(match), "\n")
  print(f'Statistically ML: {float(overall_win_model.predict(input_predict)):.3f}%, Implied odds: {float(1/(overall_win_model.predict(input_predict)/100)):.3f}')
  print(f'Statistically O2.5: {float(overall_ou_model.predict(input_predict_ou)):.3f}%, Implied odds: {float(1/(overall_ou_model.predict(input_predict_ou)/100)):.3f}')
  print(f'Statistically U2.5: {float(100-overall_ou_model.predict(input_predict_ou)):.3f}%, Implied odds: {float(1/((100-overall_ou_model.predict(input_predict_ou))/100)):.3f}')


In [ ]:
print(np.sort(epl_1819.HomeTeam.unique()))

In [ ]:
##Norway

In [ ]:
final_output('Mjondalen', 'Molde')

In [ ]:
final_output('Kristiansund', 'Lillestrom')

In [ ]:
final_output('Odd', 'Bodo/Glimt')

In [ ]:
final_output('Rosenborg', 'Stabaek')

In [ ]:
final_output('Brann', 'Stromsgodset')

In [ ]:
final_output('Tromso', 'Sarpsborg 08')

In [ ]:
final_output('Valerenga', 'Sandefjord')

In [ ]:
final_output('Haugesund', 'Viking')

In [ ]:
#MLS

In [ ]:
final_output('Philadelphia Union', 'Portland Timbers')

In [ ]:
final_output('Seattle Sounders', 'Austin FC')

In [ ]:
final_output('Cruz Azul', 'Santos Laguna')